In [1]:
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)

from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

1.0.10
Turning diagnostics collection on. 


In [2]:
from azure.common.credentials import ServicePrincipalCredentials
from azureml.core.authentication import ServicePrincipalAuthentication

TENANT_ID = 'x-x-x-x-x'
CLIENT = 'x-x-x-x-x'
KEY = 'x-x-x-x-x'

auth = ServicePrincipalAuthentication(TENANT_ID, CLIENT, KEY)

In [1]:
ws = Workspace.from_config(auth=auth)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "dummycluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=1)
    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found existing compute target


In [5]:
from azureml.train.dnn import TensorFlow

script_params={
}

# NOT!    pip_packages = ['horovod']    # NOT!
pip_packages = []

In [6]:
estimator= TensorFlow(source_directory='dummytrain',
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='99-ms-train-dummy.py',
                      node_count=1,
                      process_count_per_node=1,
                      distributed_backend='mpi',
                      pip_packages = pip_packages,
                      use_gpu=True)


In [7]:
from azureml.core import Experiment

experiment_name = 'dummy-error'
experiment = Experiment(ws, name=experiment_name)
run = experiment.submit(estimator)
print(run)

Run(Experiment: dummy-error,
Id: dummy-error_1549403985_29f96d06,
Type: azureml.scriptrun,
Status: Starting)


In [8]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

In [9]:
rd = RunDetails(run)

In [2]:
rd.get_widget_data()